# Analysis of some Paleoclimate model data

*Paolo Davini (2024)*

This notebook aims at illustrating a simple analysis of paleoclimate simulations, part of the [PMIP](https://pmip.lsce.ipsl.fr/) project, that probably many of you already know much better than me! We take data from one specific climate model, namely the AWI-ESM-1 Earth System Model. To simplify the analysis and increase the amount of data available we interpolated everything to 2x2 degree grid, and perform climatological mean so that you will have only one record per variable! Some operations indeed can be computationally heavy, and we cannot run them on a cloud jupyter notebook :-) 

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## Data Loading

We have data from the historical CMIP6 simulations, together data from the Last Glacial Maximum (LGM) and the Last Integracial (LIG). A few variables are present in the different simulations, ranging from surface winds to zonal winds

In [ ]:
historical = xr.open_mfdataset('ussp/*AWI*historical*.nc', compat='override',  join="override")
historical

In [ ]:
lgm = xr.open_mfdataset('ussp/*AWI*lgm*.nc', compat='override', join="override")
lgm

In [ ]:
lig = xr.open_mfdataset('ussp/*AWI*lig*.nc', compat='override',  join="override")
lig

## Basic comparison

A simple way to see the difference in ice sheet extension can go through the surface pressure!

In [ ]:
pp = (lgm['ps']-historical['ps']).plot(subplot_kws=dict(projection=ccrs.PlateCarree()))
pp.axes.coastlines()

Why **the surface pressure is so different between historical and LGM in some specific area of the globe**?

With a little bit more of coding handling, we can plot the differences between the precipitation field in a multiplot adding the coastlines!

In [ ]:
fig, axs = plt.subplots(3,1, figsize=(15,10), subplot_kw=dict(projection=ccrs.PlateCarree()))
(lgm['pr']*86400).plot(ax=axs[0],vmax=15) #converting from Kgm-1s-1 to mm/day
axs[0].set_title('LGM')
axs[0].coastlines()
(historical['pr']*86400).plot(ax=axs[1],vmax=15)
axs[1].set_title('historical')
axs[1].coastlines()
((historical['pr']-lgm['pr'])*86400).plot(ax=axs[2])
axs[2].set_title('DELTA')
axs[2].coastlines()

## Writing a function to speed up analysis!

The above type of plot can be very helpful to replicate, we can build a function to do it so that we avoid to write all that bunch of code every time

In [ ]:
def compare_plot(field1, field2, vmin=None, vmax=None):
    
    """Small function that plot two fields and their difference"""
    
    fig, axs = plt.subplots(2,1, figsize=(10,10), subplot_kw=dict(projection=ccrs.PlateCarree()))
    field1.plot(ax=axs[0],vmin=vmin, vmax=vmax)
    axs[0].set_title(field1.name)
    axs[0].coastlines()
    #field2.plot(ax=axs[1],vmin=vmin, vmax=vmax)
    #axs[1].set_title(field2.name)
    #axs[1].coastlines()
    (field1-field2).plot(ax=axs[1],cmap='seismic')
    axs[1].set_title('DELTA')
    axs[1].coastlines()

In [ ]:
compare_plot(lig['tas'],historical['tas'],vmin=200,vmax=310)

## Circulation changes
One quite interesting feature to study is to check how the wind circulation is changing, and this can be done exploring the zonal wind `ua`, which is available in multiple experiments. However, we need to take a bit of confidence with those new field, since it is stored not only at the surface but on the entire atmospheric column following *pressure levels*. 

In [ ]:
historical['ua']

The coordinate can be accessed easily with xarray, and it shows that the data has 19 levels, stored on pressure and the unit is Pa

In [ ]:
historical.plev

With a bit of caution, we can run the same function as before. What do you spot here? Are the jet streams very different? 

In [ ]:
compare_plot(lgm['ua'].sel(plev=85000), historical['ua'].sel(plev=85000))

We can even do something more complex, still with the feature of xarray. We use the matplotlib `countourf` function to produce the zonal mean of the wind, in order to see the cross section of zonal winds!

In [ ]:
zonal_hist = (historical['ua'].mean(dim='lon'))
zonal_lgm = lgm['ua'].mean(dim='lon')
fig, ax = plt.subplots()
zonal_hist.squeeze().plot.contourf(levels=15)
ax.invert_yaxis()


Similarly with what done above, we can run the difference between the two fields, and even play with color palette `cmap`. For a complete list of possible palettes, check the [matplotlib website](https://matplotlib.org/stable/users/explain/colors/colormaps.html)

In [ ]:
fig, ax = plt.subplots()
(zonal_hist-zonal_lgm).squeeze().plot.contourf(levels=15, cmap='PRGn')
ax.invert_yaxis()

## Play with the data!

You can inspect freely the data provided and analyse the climatological differences
- What are the differences in surface winds? Are they comparable with the one you see at zonal winds?
- Are the many differences in cloud cover distribution between the different experiments?
- Can you load ERA5 temperature, and estimate the temperature differences between the climate models in the different experiments and the present climate observations?
- What about the climate over Italy in the different experiments? 